### JIM XU
### 831-156-383

# A3: A\*, IDS, and Effective Branching Factor

For this assignment, implement the Recursive Best-First Search
implementation of the A\* algorithm given in class.  Name this function `aStarSearch`.  Also in this notebook include your `iterativeDeepeningSearch` functions.  Define a new function named `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

Apply `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. For this you must include your implementations of these functions, from Assignment 2:

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,
  
plus the following function for the eight-tile puzzle:

  * `goalTestF_8p(state, goal)`
  
Compare their results by displayng
solution path depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  Do this by defining the following function that prints the table as shown in the example below.

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])
   
Define this function so it takes any number of $h$ functions in the list that is the fourth argument.

## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

Insert your functions here.

In [1]:
"""
    functions for the 8-puzzle *
"""
# printState_8p(state, special=""), friendly printing the state in readable format
def printState_8p(state, special=""):
    newLine = 0
    print(special, end="")
    for num in state:
        if newLine == 3:
            newLine = 0
            print()
            print(special, end="")
        print(" " if num == 0 else num, end=" ")
        newLine += 1
    print()
    
# findBlank_8p(state): return the row and column index for the location of the a Blank
def findBlank_8p(state):
    index = state.index(0)
    # a is the row while b is the col
    a = (int)(index / 3)
    b = index % 3
    return a, b

# actionsF_8p(state): returns a list of up to four valid actions that can be applied in state. 
# Return them in the order left, right, up, down, though only if each one is a valid action.
def actionsF_8p(state):
    actions = []
    a, b = findBlank_8p(state)
    # check the validation of possible actions 
    if b > 0: actions.append(("left", 1))
    if b < 2: actions.append(("right", 1))
    if a > 0: actions.append(("up", 1))
    if a < 2: actions.append(("down", 1))
    return actions

# takeActionF_8p(state, action): return the state that results from applying action in state.
def takeActionF_8p(state, action):
    # make a new value copy of original state
    newState = state[:]
    _a = 0
    _b = 0
    # calculate the increments of row and col
    if action[0] == "left": _b = -1
    if action[0] == "right": _b = 1
    if action[0] == "up": _a = -1
    if action[0] == "down": _a = 1
    a, b = findBlank_8p(newState)
    # calculate the new row and col
    _a += a
    _b += b
    # incase not valid action
    # discard the action and return the original state
    if not 0<= _a < 3: 
        print("invalid action")
        return state
    if not 0<= _b < 3:
        print("invalid action")
        return state
    # switch the element
    newState[_a*3+_b], newState[a*3+b] = newState[a*3+b], newState[_a*3+_b]
    return (newState, action[1])

def goalTestF_8p(state, goalState):
    return state == goalState

In [2]:
"""
    h(n) an estimate of the sum of the remaining step costs to a goal
"""
import math

# negative way, make no prediction on the future cost to the goalState
def h1_8p(state, goal):
    return 0

# naive way, take Manhattan distance between the currentState and goalState
def h2_8p(state, goal):  
    x1, y1 = findBlank_8p(state)
    x2, y2 = findBlank_8p(goal)
    return abs(x1-x2) + abs(y1-y2)
    
# simple way, consider the moves to restore all the numbers to correct position
# while for each move, we can at most minimize two dislocated numbers
# and the return value gives the at least number of moves to restore the correct location for all numbers
def h3_8p(state, goal):
    step = 0
    for i in range(0, 9):
        step += locate_i(state, goal, i, 3)
    return int(step / 2)
    
# Reloacte i(loc) to its correct position
def locate_i(state, goal, loc, rcNum):
    num = state[loc]
    des = goal.index(num)
    return abs(des - loc) % rcNum + int(abs(des - loc) / rcNum)

In [3]:
# from A3mysolution import *
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            # Add startState to front of solution path, in result, returned by depthLimitedSearch
            # if find an path, then it must cost the minimum steps to reach
            # and then there is no need for futher iteration, return the path
            result.insert(0, startState)
            return result
    return 'cutoff'

def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    # use the goal value to store the steps
    global goalStep
    if state == goalState:
        return []
    if depthLimit == 0:
        # Return the string 'cutoff' to signal that the depth limit was reached
        return 'cutoff'
    cutoffOccurred = False
    # consider each action of state
    for action in actionsF(state):
        goalStep += 1
        # the resulting state by taking the action
        (childState, stepCost) = takeActionF(state, action)
        # further explore the resulting state
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        # check the resulting state is valid or not
        # if reach the maximum depth, return cutoff
        if result is 'cutoff':
            cutoffOccurred = True
        # elif there is a goal state found, add the current state to path
        elif result is not 'failure':
            # Add childState to front of partial solution path, in result, returned by depthLimitedSearch
            result.insert(0, childState)
            return result
    # if reach the maximum depth or goal not found
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'

#returns number of nodes expanded and depth reached during a search.
def ebf(nNodes, depth, precision=0.01):
    """
    :name ebf: effective braching factor, 1 + b + b^2 + .. + b^d = #nodes
    :type nNodes: int -- Number of nodes that expanded
    :type depth: int -- Depth of the tree expanded
    :rtype: float --  Number of nodes expanded and depth reached during a search.
    """
    if nNodes == 0:
        return 0
    if depth == 0:
        return nNodes
    # set lowest bound to 0 (could be improved)
    lowest_b = 0
    # set highest bound to nNodes**(depth**-1)
    # suppose we have only b*^d = n, neglecting 1 + b + b^2 + ... + b^(d-1)
    # the b value we get is the highest bound of b
    highest_b = nNodes**(depth**-1)
    # set b to the highest bound
    b = highest_b
    while True:
        # calculate the score    
        score = calEBFscore(b, depth)
        # if score in precision range, done
        if nNodes - precision <= score <= nNodes + precision:
            break
        # if score is higher than expected, cut b to the average of lowest bound and b
        if score > nNodes + precision:
            highest_b = b
            b = (lowest_b + b) / 2
        # if score is lower than expected, cut b to the average of highest bound and b
        else:
            lowest_b = b
            b = (b + highest_b) / 2
    return b
    
# calculate the value of 1 + b + b^2 + ... + b^d 
# handle b == 1 condition
def calEBFscore(b, depth):
    if b == 1:
        return depth + 1
    else:
        return (1 - b**(depth + 1))/(1 - b)

In [4]:
# aStarSearch.py
# Recursive Best First Search (Figure 3.26, Russell and Norvig)
# Recursive Iterative Deepening form of A*, where depth is replaced by f(n)

# MODIFIED FROM DR.ANDERSON VERSION
# From [http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/07%20Informed%20Search.ipynb]

class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    # use the goal value to store the steps
    global goalStep
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        goalStep += 1
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":
            result.insert(0,parentNode.state)
            return (result, bestChild.f)

In [5]:
import sys

# declare a global value
goalStep = 0

# runExperiment and compare output
def runExperiment(goalState1, goalState2, goalState3, hfunctions):
    global goalStep
    # hard-code startState, although not suggested
    startState = [1,  2,  3,  4,  0,  5,  6,  7,  8]
    goalStates = [goalState1, goalState2, goalState3]
    # hard-code maxDepth
    maxDepth = sys.maxsize
    # hard-code actionsF, takeActionF
    for goalState in goalStates:
        goalStep = 0
        # print titile : goalState
        print(goalState)
        # print column names    
        print('{:^15}\t{}\t{}\t{}\t'.format("Algorithm", "Depth", "Nodes", "EBF"))
        result = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, maxDepth)
        # print IDS results
        print('{:^15}\t{}\t{}\t{:.3f}\t'.format("IDS", len(result)-1, goalStep, ebf(goalStep, len(result)-1)))
        for i, hf in enumerate(hfunctions):
            goalStep = 0
            (result, depth) = aStarSearch(startState, actionsF_8p, takeActionF_8p,
                        lambda s: goalTestF_8p(s, goalState),
                        lambda s: hf(s, goalState))
            # print A* results
            print('{:^15}\t{}\t{}\t{:.3f}\t'.format("A*h"+str(i+1), depth, goalStep, ebf(goalStep, depth)))
        print()
    

In [6]:
runExperiment([1, 2, 3, 4, 0, 5, 6, 7, 8],[1, 2, 3, 4, 5, 8, 6, 0, 7],[1, 0, 3, 4, 5, 8, 2, 6, 7],[h1_8p, h2_8p, h3_8p])

[1, 2, 3, 4, 0, 5, 6, 7, 8]
   Algorithm   	Depth	Nodes	EBF	
      IDS      	0	0	0.000	
     A*h1      	0	0	0.000	
     A*h2      	0	0	0.000	
     A*h3      	0	0	0.000	

[1, 2, 3, 4, 5, 8, 6, 0, 7]
   Algorithm   	Depth	Nodes	EBF	
      IDS      	3	43	3.086	
     A*h1      	3	116	4.488	
     A*h2      	3	51	3.297	
     A*h3      	3	9	1.578	

[1, 0, 3, 4, 5, 8, 2, 6, 7]
   Algorithm   	Depth	Nodes	EBF	
      IDS      	11	225850	2.954	
     A*h1      	11	643246	3.263	
     A*h2      	11	100046	2.733	
     A*h3      	11	8008	2.138	



I **tried to use pandas** but I found **it is not convinient** than simply using print statement. I generated similar output as required, as above. It is readable enough so I didn't make any further polishment.

First, some example output for the ebf function.

In [7]:
ebf(10, 3)

1.6600087601905824

In [8]:
ebf(3, 1)

1.9921875

In [9]:
ebf(1, 1)

0.0078125

1 + b^1 = 1

In [10]:
ebf(2, 2)

0.6187184335382292

1 + b^1 + b^2 = 2

In [11]:
ebf(3, 3)

0.8112653832979171

1 + b^1 + b^2 + b^3 = 3

The smallest argument values should be a depth of 0, and 1 node.

In [12]:
ebf(1, 0)

1

In [13]:
ebf(2, 1)

1.0

In [14]:
ebf(2, 1, precision=0.000001)

1.0

In [15]:
ebf(200000, 5)

11.27559688043982

In [16]:
ebf(200000, 50)

1.2348192485141916

Self-Testing for some examples

In [17]:
actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

[('left', 1), ('right', 1), ('up', 1)]

In [18]:
takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], ("up", 1))

([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

In [19]:
goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

True

In [20]:
iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                                 actionsF_8p, takeActionF_8p, 10)

'cutoff'

In [21]:
print("-- from --")
printState_8p([1, 2, 3, 6, 5, 0, 7, 8, 4])
print("-- to --")
printState_8p([1, 2, 3, 6, 5, 4, 7, 8, 0])
print("needs at least ", h3_8p([1, 2, 3, 6, 5, 0, 7, 8, 4], [1, 2, 3, 6, 5, 4, 7, 8, 0]), " steps")

-- from --
1 2 3 
6 5   
7 8 4 
-- to --
1 2 3 
6 5 4 
7 8   
needs at least  1  steps


The difference above is 4 and 0, we need to take 1 cost moving 4 to where 0 stands, and 1 cost moving 0 to where 4 stands in order to reach the goalState. We call the current state has **chaos value equals 4**. Whereas, we can **diminish at most 2 chaos value by making one move(sometimes could improve chaos value)**, so there are at least 1 step for cancelling chaos value. And the true cost from originalState to goalState is 1, obviously.

In [22]:
print("-- from --")
printState_8p([1, 2, 3, 4, 5, 6, 7, 8, 0])
print("-- to --")
printState_8p([1, 2, 3, 6, 5, 4, 7, 8, 0])
print("needs at least ", h3_8p([1, 2, 3, 4, 5, 6, 7, 8, 0], [1, 2, 3, 6, 5, 4, 7, 8, 0]), " steps")

-- from --
1 2 3 
4 5 6 
7 8   
-- to --
1 2 3 
6 5 4 
7 8   
needs at least  2  steps


The difference above is 4 and 6, we need to take 2 cost moving 4 to where 6 stands, and 2 cost moving 6 to where 4 stands. The total chaos value is 4. Whereas, we can diminish chaos value by 2 by making one move, so there are **at least 2 steps** for cancelling all chaos value. While the **true cost** from originalState to goalState **is apparent more than 2**.

In [23]:
print("-- from --")
printState_8p([5, 2, 8, 0, 1, 4, 3, 7, 6])
print("-- to --")
printState_8p([0, 2, 3, 1, 4,  6, 7, 5, 8])
print("needs at least ", h3_8p([5, 2, 8, 0, 1, 4, 3, 7, 6], [0, 2, 3, 1, 4,  6, 7, 5, 8]), " steps")

-- from --
5 2 8 
  1 4 
3 7 6 
-- to --
  2 3 
1 4 6 
7 5 8 
needs at least  6  steps


Here is a simple example using our usual simple graph search.

In [24]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [25]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [26]:
takeActionF_simple('a', actions[0])

('b', 1)

In [27]:
goalTestF_simple('a', 'a')

True

In [28]:
h_simple('a', 'z')

1

In [29]:
depthLimitedSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['b', 'a', 'b', 'a', 'c', 'h', 'i', 'k', 'z']

In [30]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [31]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [32]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            